In [38]:
import re
import pandas as pd
import csv
import string
from tkinter import *
from tkinter.filedialog import askopenfilename
import time
import os
import urllib
from pathlib import Path
# Uncomment the next two lines if you want to see expanded output
#pd.options.display.max_colwidth
#pd.set_option('display.max_colwidth', -1)

###Select the text you want to upload
This will open a file picker in a new window

In [39]:
root = Tk()
root.withdraw()
file_path = askopenfilename()
f = open(file_path)
text = f.read()
root.update()
root.destroy()

###Select the exsisting annotations you want to upload
This will open a file picker in a new window

In [40]:
root = Tk()
root.withdraw()
csv_file_path = askopenfilename()
df = pd.read_csv(csv_file_path)
root.update()
root.destroy()

###Find downloads folder windows, linux, mac

In [41]:
if os.name == 'nt':
    import ctypes
    from ctypes import windll, wintypes
    from uuid import UUID

    # ctypes GUID copied from MSDN sample code
    class GUID(ctypes.Structure):
        _fields_ = [
            ("Data1", wintypes.DWORD),
            ("Data2", wintypes.WORD),
            ("Data3", wintypes.WORD),
            ("Data4", wintypes.BYTE * 8)
        ] 

        def __init__(self, uuidstr):
            uuid = UUID(uuidstr)
            ctypes.Structure.__init__(self)
            self.Data1, self.Data2, self.Data3, \
                self.Data4[0], self.Data4[1], rest = uuid.fields
            for i in range(2, 8):
                self.Data4[i] = rest>>(8-i-1)*8 & 0xff

    SHGetKnownFolderPath = windll.shell32.SHGetKnownFolderPath
    SHGetKnownFolderPath.argtypes = [
        ctypes.POINTER(GUID), wintypes.DWORD,
        wintypes.HANDLE, ctypes.POINTER(ctypes.c_wchar_p)
    ]

    def _get_known_folder_path(uuidstr):
        pathptr = ctypes.c_wchar_p()
        guid = GUID(uuidstr)
        if SHGetKnownFolderPath(ctypes.byref(guid), 0, 0, ctypes.byref(pathptr)):
            raise ctypes.WinError()
        return pathptr.value

    FOLDERID_Download = '{374DE290-123F-4565-9164-39C4925E467B}'

    def get_download_folder():
        return _get_known_folder_path(FOLDERID_Download)
else:
    def get_download_folder():
        home = os.path.expanduser("~")
        return os.path.join(home, "Downloads")

###Get position of annotations in text (Mat take a while for large files). Will be in downloads folder

In [46]:
'''
If total is off uncomment lines to check why
'''
root, ext = os.path.splitext(Path(file_path).name)
name = get_download_folder()
name = name + "/" + root + '.csv'
#TODO change 1522.csv to use identifier
#p = urllib.parse.urljoin(name, '1522.csv')
count = 0
with open(name, "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    header = ["Segment", "Start", "End", "Occurs", "Concept"]
    writer.writerow(header)
    url =  'https://amphora.asu.edu/amphora/resource/' + root
    for index, row in df.iterrows():
        l = []
        seg = row['Segment']
        seg = re.sub(r'\(', r'\\(', seg ) #Need to escape paraenthesis due to regex
        seg = re.sub(r'\)', r'\\)', seg )
        seg = re.sub(r'\?', r' ', seg )
        seg = re.sub(r'[^\x00-\x7F]+',' ', seg)
        seg = re.sub(r'\s+', ' ', seg)
        seg = re.sub(r'\\r', ' ', seg)
        m = re.search(seg, text, re.M | re.DOTALL)
        #print(seg)
        if m:
            found = m.group(0)
            l.append(str(found))
            l.append(str(m.start()))
            l.append(str(m.end()))
            l.append(url)
            l.append( row['Code'])
            writer.writerow(l)
            #print(m.start(), m.end())
            #print("FOUND")
            #print(found)
            count += 1
        else:
            seg = re.sub(r'\W+', '\\W+', seg)
            m = re.search(seg, text, re.M | re.DOTALL)
            if m:
                pattern = re.compile(r'1522.txt')
                found = m.group(0)
                l.append(str(found))
                l.append(str(m.start()))
                l.append(str(m.end()))
                l.append(url)
                l.append( row['Code'])
                writer.writerow(l)
                #print(m.start(), m.end())
                #print("FOUND")
                #print(found)
                count += 1

            
        #print("***********************")
print("The total number of matches is {}".format(count))

The total number of matches is 24


In [44]:
csv_file_path

'/Users/taylorquinn/Desktop/MAXQDA.csv'